In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/videodataset/video1.mp4
/kaggle/input/videodataset/video2.mp4


In [2]:
# !pip install filterpy deep-sort-realtimea

In [3]:
import torch
import torchvision
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

# YoloV5 Network and DeepSort

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-22 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [5]:
def detect_cars(img, model, threshold = 0.5):
    detections = []
    pred = model(img)
    car_pred = pred.pred[0][pred.pred[0][:, 5] == 2]
    boxes = car_pred[:, :4].cpu().numpy()
    confidences = car_pred[:, 4].cpu().numpy()
    class_label = car_pred[:, 5].cpu().numpy()

    for i in range(len(class_label)):
        if(confidences[i] > threshold):
            cv2.rectangle(frame, (int(boxes[i][0]), int(boxes[i][1])), (int(boxes[i][2]), int(boxes[i][3])), (255, 255, 0), 2)
            cv2.putText(frame, f"Car : {confidences[i]:.2f}", (int(boxes[i][0])-10, int(boxes[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
            detections.append((boxes[i], confidences[i]))

    return detections

In [6]:
def update_track(tracks, count, track_ids):
    for t in tracks:
        if not t.is_confirmed():
            continue
        t_id = t.track_id
        if(t_id not in track_ids):
            count += 1
            track_ids.append(t_id)

        ltrb = t.to_ltrb()
        bbox = ltrb

    return track_ids, count

In [7]:
for i in range(1, 3):
    object_tracker = DeepSort(max_iou_distance=0.3, max_age=30, nms_max_overlap = 0.5)    
    cap = cv2.VideoCapture(f'/kaggle/input/videodataset/video{i}.mp4')

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    result = cv2.VideoWriter(f'YOLODeepSortOutput{i}.mp4', fourcc, 25.0, (int(cap.get(3)), int(cap.get(4))))

    track_ids = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_cars(frame, model, 0.6)
        tracks = object_tracker.update_tracks(detections, frame=frame)
        track_ids, count = update_track(tracks, count, track_ids)
        cv2.putText(frame, f'CAR COUNT: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (255,255,0), 3)

        result.write(frame)

    cap.release()
    result.release()
    
    print(count)


17
25
